# Imports and setting up viz

NB : conda env1 on Mac, lam1env on spirit (Python3.12)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

#import personnal tools
import sys
sys.path.append('../../python_tools/')
from tools import *

In [72]:
rivers = cfeature.NaturalEarthFeature('physical', 'rivers_lake_centerlines', '10m',edgecolor=(0, 0, 0, 0.3), facecolor='none')

# Load files

LAM output analysis.

Simu irr et simu no-irr.

## Sims

In [73]:
noirr_dir='../../../JZ_simu_outputs/LAM/LAM_1000_40/noirr_2010_2022'
irr_dir='../../../JZ_simu_outputs/LAM/LAM_1000_40/irr_2010_2022'

In [ ]:
#open netcdf files for native routing on ORC grid
# filename = '{}/SRF/MO/*history.nc'.format(noirr_dir)
filename = '{}/SRF/TS_MO_noirr*.nc'.format(noirr_dir)

ORCnoirr0 = xr.open_mfdataset(filename)
ORCnoirr = ORCnoirr0.rename({'time_counter':'time'})
ORCnoirr.attrs['name'] = 'no_irr'
ORCnoirr = ORCnoirr.sel(lon=slice(-13,6),lat=slice(32,49))
ORCnoirr

In [ ]:
filename = '{}/SRF/MO/*history.nc'.format(irr_dir)
# filename = '{}/SRF/TS_MO_irr*.nc'.format(irr_dir)

ORCirr0 = xr.open_mfdataset(filename)
ORCirr = ORCirr0.rename({'time_counter':'time'})
ORCirr.attrs['name'] = 'irr'
ORCirr = ORCirr.sel(lon=slice(-13,6),lat=slice(32,49))

# #manually define irrig_deficit as netirrig-irrigation in ORC file
# ORCirr['irrig_deficit'] = ORCirr['netirrig'] - ORCirr['irrigation']
# #make irrig_deficit units mm/day
# ORCirr['irrig_deficit'].attrs['units'] = 'mm/day'
# ORCirr['irrig_frac'] = ORCirr['irrigmap_dyn']/ORCirr['Areas']

ORCirr0

## Obs

In [76]:
load_Ebro_obs=True

In [77]:
if (load_Ebro_obs):
    filename = '../../../obs/SM_based_inversion_approach_EBRO_Irrigation.nc'
    irr_Ebro = xr.open_dataset(filename)
    irr_Ebro = irr_Ebro.rename({'Time':'time'})
    irr_Ebro = irr_Ebro.rename({'Longitude':'lon'})
    irr_Ebro = irr_Ebro.rename({'Latitude':'lat'})
    irr_Ebro = irr_Ebro.rename({'Irrigation':'irrigation'})
    #make lon and lat coordinates and not variables
    # irr_Ebro = irr_Ebro.set_coords(['lon','lat'])

    irr_Ebro = regrid_to_lon_lat(irr_Ebro)
    irr_Ebro.attrs['name'] = 'obs_Ebro'

    #convert to mm/day
    irr_Ebro['irrigation'] = irr_Ebro['irrigation']/30
    irr_Ebro['irrigation'].attrs['units'] = 'mm/day'
    
    #resample irr_Ebro to monthly values
    irr_Ebro = irr_Ebro.resample(time='1M').mean()
    
    irr_Ebro

In [ ]:
#resample irr_Ebro to monthly values
irr_Ebro = irr_Ebro.resample(time='1M').mean()
irr_Ebro

## Period selection

In [79]:
#select only JJA
# ORCnoirr = ORCnoirr.sel(time=ORCnoirr['time.season']=='JJA')
# ORCirr = ORCirr.sel(time=ORCirr['time.season']=='JJA')
# newORC = newORC.sel(time=newORC['time.season']=='JJA')

In [80]:
#match the period of Ebro data
date_min='2016-01-01'
date_max='2020-08-31'

ORCnoirr = ORCnoirr.sel(time=slice(date_min,date_max))
ORCirr = ORCirr.sel(time=slice(date_min,date_max))
irr_Ebro = irr_Ebro.sel(time=slice(date_min,date_max))

## Masks

In [81]:
#create a masks of data points where irrig_frac >5%
irr_mask = (ORCirr['irrigmap_dyn']/ORCirr['Areas'])>0.05

In [82]:
mORCirr=ORCirr.where(irr_mask)
mORCirr.attrs['name'] = 'irr'

mORCnoirr=ORCnoirr.where(irr_mask)
mORCnoirr.attrs['name'] = 'no_irr'

In [83]:
con_mask=ORCnoirr['Contfrac']>0.9
ip_mask=polygon_to_mask(ORCnoirr, iberic_peninsula)

In [84]:
ip_ORCnoirr=ORCnoirr.where(con_mask).where(ip_mask['mask'])
ip_ORCirr=ORCirr.where(con_mask).where(ip_mask['mask'])

In [85]:
ebro_mask = irr_Ebro['irrigation']>0

In [86]:
ORCirr_iObsEbro = ORCirr.interp_like(irr_Ebro)
ebro_ORCirr_iObsEbro = ORCirr_iObsEbro.where(ebro_mask)

In [103]:
obsEbro_iORCirr = irr_Ebro.interp_like(ORCirr)
ebro_mask_iORCirr = obsEbro_iORCirr['irrigation']>0
ebro_ORCirr = ORCirr.where(ebro_mask_iORCirr)

# Maps

In [ ]:
ds=obsEbro_iORCirr
var='irrigation'
vmin=0
vmax=0.8
map_ave(ds, var, vmin=vmin, vmax=vmax, cmap=wet)

In [ ]:
ds=ebro_ORCirr
var='irrigation'
vmin=0
vmax=0.8
title='Irrigation (mm/day)'
map_ave(ds, var, vmin=vmin, vmax=vmax, title=title, cmap=wet, hex=False)

In [ ]:
#diff
ds1=ebro_ORCirr
ds2=obsEbro_iORCirr
var='irrigation'
vmax=0.8
vmin=-vmax
map_diff_ave(ds1, ds2, var, vmin=vmin, vmax=vmax)

In [ ]:
#diff with significance
ds1=ip_ORCirr   
ds2=ip_ORCnoirr
var='streamr'
vmax=45
vmin=-vmax
map_diff_ave(ds1, ds2, var, vmin=vmin, vmax=vmax, sig=True)

In [ ]:
#relative difference
ds1=ip_ORCirr
ds2=ip_ORCnoirr
var='streamr'
vmax=50
vmin=-50
map_rel_diff_ave(ds1, ds2, var, vmin=vmin, vmax=vmax)

In [ ]:
#Display map of var for ds on restricted area
var='irrigation'
ds = ORCirr
latmin=39.75
latmax=44.05
lonmin= -4.5
lonmax= 2.5
# latmin=41.
# latmax=42.5
# lonmin= 0.
# lonmax= 1.5

#restrict ds to latmin, latmax, lonmin, lonmax
ds = ds.where(ds.lat >= latmin, drop=True).where(ds.lat <= latmax, drop=True)
ds = ds.where(ds.lon >= lonmin, drop=True).where(ds.lon <= lonmax, drop=True)
plotvar = ds[var].mean(dim='time')
map_plotvar(plotvar, cmap=wet, vmin=0, vmax=0.8, title='Applied irrigation (mm/day)')

In [ ]:
#Display map of var for ds on restricted area
var='irrigation'
ds1 = ORCirr
ds2 = obsEbro_iORCirr
latmin=39.75
latmax=44.05
lonmin= -4.5
lonmax= 2.5
# latmin=41.
# latmax=42.5
# lonmin= 0.
# lonmax= 1.5

#restrict ds to latmin, latmax, lonmin, lonmax
ds1 =ds1.where(ds1.lat >= latmin, drop=True).where(ds1.lat <= latmax, drop=True)
ds1 =ds1.where(ds1.lon >= lonmin, drop=True).where(ds1.lon <= lonmax, drop=True)
ds2 =ds2.where(ds2.lat >= latmin, drop=True).where(ds2.lat <= latmax, drop=True)
ds2 =ds2.where(ds2.lon >= lonmin, drop=True).where(ds2.lon <= lonmax, drop=True)

map_diff_ave(ds1, ds2, var, vmin=vmin, vmax=vmax)

# Time series

In [ ]:
var='irrigation'

ds1=ebro_ORCirr
ds2=obsEbro_iORCirr
ds3=irr_Ebro
ds_list=[ds1, ds2]
# ds_list=[ds2]

year_min=2016
year_max=2020
time_series_ave(ds_list, var, year_min=year_min, year_max=year_max, title=var)
seasonal_cycle_ave(ds_list, var, year_min=year_min, year_max=year_max, title=var)

In [ ]:
seasonal_cycle_ave(ds_list, 'precisol', title=var)

In [ ]:
seasonal_cycle_ave(ds_list, 'snow', title=var)